<a href="https://colab.research.google.com/github/JaeminKIM-Irene/FC_AI_ML_Project/blob/main/irenekim/ML_Project_LightGBM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 설치에 필요한 라이브러리들이 있다면 모두 적어둡니다. anaconda에 기본적으로 설치되지 않은 라이브러리들을 적어두세요.
!pip install lightgbm optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.5 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from lightgbm import LGBMClassifier

from sklearn.model_selection import KFold

import optuna

from sklearn.model_selection import StratifiedKFold
from functools import partial

from sklearn.metrics import roc_auc_score

In [4]:
# google drive mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# read data
result_list = []
base_url = '/content/drive/MyDrive/Colab Notebooks/data/playground-series-s3e23/'
train = pd.read_csv(base_url+'train.csv', index_col='id')
test = pd.read_csv(base_url+'test.csv', index_col='id')
submission = pd.read_csv(base_url+'sample_submission.csv', index_col='id')

with pd.option_context('display.min_rows', 6) :
  display(train)

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
id,,,,,,,,,,,,,,,,,,,,,
0,22.0,3.0,1.0,2.0,60.0,278.63,0.06,19.56,14.25,5448.79,...,17,1,1,0,16.0,9.0,38.0,22.0,5.0,False
1,14.0,2.0,1.0,2.0,32.0,151.27,0.14,7.00,21.11,936.71,...,11,0,1,0,11.0,11.0,18.0,14.0,3.0,False
2,11.0,2.0,1.0,2.0,45.0,197.65,0.11,8.05,22.76,1754.01,...,8,0,1,0,12.0,11.0,28.0,17.0,3.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101760,26.0,1.0,1.0,1.0,83.0,360.17,0.04,22.75,12.56,5893.69,...,20,0,4,0,10.0,8.0,51.0,28.0,1.0,False
101761,10.0,2.0,1.0,2.0,43.0,191.76,0.15,6.46,29.53,1534.04,...,7,0,1,0,11.0,14.0,24.0,19.0,3.0,False
101762,136.0,18.0,18.0,1.0,296.0,1704.57,0.02,44.82,44.08,77011.02,...,102,1,11,0,22.0,31.0,162.0,135.0,35.0,False


In [6]:
# PCA

In [7]:
# data split for baseline model
from sklearn.model_selection import train_test_split

X = train.drop(columns=['defects'])
y = train.defects

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.05, random_state=61)

In [8]:
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

(96674, 21) (96674,) (5089, 21) (5089,)


## Baseline Model

In [ ]:
model = LGBMClassifier()

model.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 21922, number of negative: 74752
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016423 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3581
[LightGBM] [Info] Number of data points in the train set: 96674, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.226762 -> initscore=-1.226685
[LightGBM] [Info] Start training from score -1.226685


LGBMClassifier()

In [ ]:
print('======Prediction======')
pred_train = model.predict(X_train)
pred_val = model.predict(X_val)

train_score = roc_auc_score(y_train, pred_train)
val_score = roc_auc_score(y_val, pred_val)

print("Train Score : %.4f" % train_score)
print("Test Score : %.4f" % val_score)

======Prediction======
Train Score : 0.6760
Test Score : 0.6630


## Optuna

In [35]:
def optimizer(trial, X, y, K) :
  num_leaves = trial.suggest_categorical('num_leaves', [64, 128, 256, 512])
  max_depth = trial.suggest_int('max_depth', 10, 25)
  learning_rate = trial.suggest_float('learning_rate', 0.001, 0.3)
  min_child_samples = trial.suggest_int('min_child_samples', 1, 100)

  model = LGBMClassifier(num_leaves=num_leaves,
                         max_depth=max_depth,
                         learning_rate=learning_rate,
                         min_child_samples=min_child_samples,
                         random_state=61)

  folds = StratifiedKFold(shuffle=True, random_state=61)
  oof = np.full(len(train), np.nan)
  auc_list = []

  for train_idx, val_idx in folds.split(X, y) :
    X_train = X.iloc[train_idx, :]
    y_train = y.iloc[train_idx]

    X_val = X.iloc[val_idx, :]
    y_val = y.iloc[val_idx]

    model.fit(X_train, y_train)

    try :
      y_val_pred = model.predict_proba(X_val)[:, 1]
    except :
      y_val_pred = model.decision_function(X_val)
    oof[val_idx] = y_val_pred
    auc = roc_auc_score(y_val, y_val_pred)
    auc_list.append(auc)

  return np.mean(auc_list)

In [33]:
# reduce boundary of learning rate
K = 5
opt_func = partial(optimizer, X=X_train, y=y_train, K=K)

study = optuna.create_study(direction='maximize')
study.optimize(opt_func, n_trials=50)

[I 2023-10-13 08:03:08,694] A new study created in memory with name: no-name-e27d37cb-51d0-4a79-90c9-b1a7d9e5633e


[LightGBM] [Info] Number of positive: 17538, number of negative: 59801
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008791 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3548
[LightGBM] [Info] Number of data points in the train set: 77339, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.226768 -> initscore=-1.226652
[LightGBM] [Info] Start training from score -1.226652
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

[I 2023-10-13 08:03:20,445] Trial 0 finished with value: 0.7875900615873597 and parameters: {'num_leaves': 128, 'max_depth': 14, 'learning_rate': 0.09830346119739063, 'min_child_samples': 55}. Best is trial 0 with value: 0.7875900615873597.


[LightGBM] [Info] Number of positive: 17538, number of negative: 59801
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008902 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3548
[LightGBM] [Info] Number of data points in the train set: 77339, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.226768 -> initscore=-1.226652
[LightGBM] [Info] Start training from score -1.226652
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

[I 2023-10-13 08:03:34,728] Trial 1 finished with value: 0.7858129291322131 and parameters: {'num_leaves': 256, 'max_depth': 14, 'learning_rate': 0.08068992464374451, 'min_child_samples': 53}. Best is trial 0 with value: 0.7875900615873597.


...

[I 2023-10-13 08:13:00,563] Trial 47 finished with value: 0.79167569819413 and parameters: {'num_leaves': 64, 'max_depth': 11, 'learning_rate': 0.04314585613337342, 'min_child_samples': 44}. Best is trial 46 with value: 0.7917584032085778.


[LightGBM] [Info] Number of positive: 17538, number of negative: 59801
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013928 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3548
[LightGBM] [Info] Number of data points in the train set: 77339, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.226768 -> initscore=-1.226652
[LightGBM] [Info] Start training from score -1.226652
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

[I 2023-10-13 08:13:21,162] Trial 48 finished with value: 0.7861074487775872 and parameters: {'num_leaves': 512, 'max_depth': 11, 'learning_rate': 0.04147659834101891, 'min_child_samples': 26}. Best is trial 46 with value: 0.7917584032085778.


[LightGBM] [Info] Number of positive: 17538, number of negative: 59801
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.018293 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3548
[LightGBM] [Info] Number of data points in the train set: 77339, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.226768 -> initscore=-1.226652
[LightGBM] [Info] Start training from score -1.226652
[LightGBM] [Info] Number of positive: 17538, number of negative: 59801
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008536 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3553
[LightGBM] [Info] Number of data points in the train set: 77339, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.226768 -> initscore=-1.226

[I 2023-10-13 08:13:31,162] Trial 49 finished with value: 0.7917677727602499 and parameters: {'num_leaves': 64, 'max_depth': 11, 'learning_rate': 0.03433900095594823, 'min_child_samples': 44}. Best is trial 49 with value: 0.7917677727602499.


In [ ]:
study.trials_dataframe()

In [12]:
print('Best Score: %.4f' % study.best_value)
print('Best Params: ', study.best_trial.params)

Best Score: 0.7920
Best Params:  {'num_leaves': 64, 'max_depth': 22, 'learning_rate': 0.0363972953785562, 'min_child_samples': 70}


In [34]:
print('Best Score: %.4f' % study.best_value)
print('Best Params: ', study.best_trial.params)

Best Score: 0.7918
Best Params:  {'num_leaves': 64, 'max_depth': 11, 'learning_rate': 0.03433900095594823, 'min_child_samples': 44}


In [13]:
optuna.visualization.plot_optimization_history(study)

In [14]:
# hyper-parameter들의 중요도
optuna.visualization.plot_param_importances(study)

## Test & Submission file

In [28]:
def oof_preds(best_model) :
  folds = StratifiedKFold(shuffle=True, random_state=61)
  final_preds = []
  auc_list = []

  for i, (train_idx, val_idx) in enumerate(folds.split(X, y)) :
    X_train = X.iloc[train_idx, :]
    y_train = y.iloc[train_idx]
    X_val = X.iloc[val_idx, :]
    y_val = y.iloc[val_idx]

    print(f"========== Fold {i+1} ==========")
    best_model.fit(X_train, y_train)
    preds = best_model.predict_proba(X_val)[:, 1]
    test_preds = best_model.predict_proba(test)[:, 1]
    final_preds.append(test_preds)
    auc = roc_auc_score(y_val, preds)

    auc_list.append(auc)

  print(f'AUC :', np.mean(auc))
  return final_preds

In [29]:
best_params = study.best_trial.params

best_model = LGBMClassifier(**best_params,
                            random_state=61)

preds = oof_preds(best_model)
preds = np.mean(preds, axis=0)

preds

========== Fold 1 ==========
[LightGBM] [Info] Number of positive: 18451, number of negative: 62959
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014115 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3547
[LightGBM] [Info] Number of data points in the train set: 81410, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.226643 -> initscore=-1.227365
[LightGBM] [Info] Start training from score -1.227365
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spl

array([0.21796356, 0.18972589, 0.68655417, ..., 0.16592192, 0.09757848,
       0.84315391])

In [30]:
submission.defects = preds

In [31]:
submission.to_csv('submission_lightGBM2.csv')